In [ ]:
import chardet
import pandas as pd

filename = 'splunk.csv'

# Step 1: Detect encoding
with open(filename, 'rb') as f:
    raw_data = f.read(100000)  # Read a chunk of the file
    result = chardet.detect(raw_data)
    detected_encoding = result['encoding']
    print(f"Detected encoding: {detected_encoding}")

# Step 2: Read the CSV using detected encoding
raw = pd.read_csv(filename, encoding=detected_encoding)

In [ ]:
df = raw
df = df.drop(columns=[
    '_raw',
    'annotations.mitre_attack.mitre_description',
    'annotations.mitre_attack.mitre_detection'
])


In [ ]:
fields_to_check = [
    'guid', 'timestamp', 'detection_type', 'name', 'severity', 
    'mitre_tactic', 'entity', 'entity_type', 'host_ip', 'source_ip',
    'dest_port', 'dst_geo', 'username', 'user_agent', 'syscall_name', 'executable',
     'message', 'proctitle'
]
for field in fields_to_check:
    if field not in df.columns:
        df[field] = ''


In [ ]:
fields_to_check = [
    'source', 'guid', 'timestamp', 'detection_type', 'name', 'severity', 'category',
    'mitre_tactic', 'entity', 'entity_type', 'host_ip', 'source_ip', 'dest_ip',
    'dest_port', 'dst_geo', 'username', 'user_agent', 'syscall_name', 'executable',
    'process', 'message', 'proctitle'
]

existing_fields = [f for f in fields_to_check if f in df.columns]
missing_fields = [f for f in fields_to_check if f not in df.columns]

print("Existing fields:")
print(existing_fields)
print("\nMissing fields:")
print(missing_fields)


In [ ]:
names = df[['AlertTitle', 'search_name', 'source', 'threat_name', 'ThreatCategory']].drop_duplicates()
pd.set_option("display.max_colwidth", None)  # Replace 50 with desired number of columns
pd.set_option("display.max_rows", 50)  # Replace 50 with desired number of columns
names

In [ ]:
users = df[['LogOnUsers', 'user', 'risk_object', 'normalized_risk_object', 'username']].drop_duplicates()
pd.set_option("display.max_rows", 100)  # Replace 50 with desired number of columns
pd.set_option("display.max_colwidth", None)  # Replace 50 with desired number of columns
users

In [ ]:
hosts = df[[ 'dest', 'dest_nt_host', 'risk_object_nt_host', 'host']].drop_duplicates()
pd.set_option("display.max_colwidth", None)  # Replace 50 with desired number of columns
hosts

In [ ]:
# Extract alert name

# Set display options
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", None)

# Step 1: Start with 'search_name' as the base for 'name'
df['name'] = df['search_name'].fillna('')

# Step 2: Append non-null values from the other fields
for field in ['AlertTitle', 'threat_name', 'ThreatCategory']:
    df['name'] = df.apply(
        lambda row: row['name'] + f"--{row[field]}" if pd.notna(row[field]) else row['name'],
        axis=1
    )
print(df['name'].unique())

In [ ]:
# Populate detection type
df['detection_type'] = df['name'].apply(
    lambda x: "Splunk Risk Threshold" if 'Risk Threshold Exceeded' in x else "Splunk Alert"
)
print(df['detection_type'].unique())

In [ ]:
df['username'] = df['user'].where(pd.notna(df['user']))
# Ensure entity and entity_type columns exist (create if missing)
df['entity'] = None
df['entity_type'] = None

# If 'host' is populated, set entity=host and entity_type='endpoint'
df.loc[df['host'].notna(), 'entity'] = df.loc[df['host'].notna(), 'host']
df.loc[df['host'].notna(), 'entity_type'] = 'endpoint'

# If 'host' is not populated, set entity=username and entity_type='user'
df.loc[df['host'].isna(), 'entity'] = df.loc[df['host'].isna(), 'username']
df.loc[df['host'].isna(), 'entity_type'] = 'user'
print(df['entity'].unique())

In [ ]:
df['timestamp'] = df['_time']
df['mitre_tactic'] = df['annotations.mitre_attack.mitre_tactic'].str.replace('\n', '  |  ', regex=False)
df['source'] = 'Splunk'


In [ ]:
# List of fields to keep
fields_to_keep = [
    'source', 'guid', 'timestamp', 'detection_type', 'name', 'severity', 'category',
    'mitre_tactic', 'entity', 'entity_type', 'host_ip', 'source_ip', 'dest_ip',
    'dest_port', 'dst_geo', 'username', 'user_agent', 'syscall_name', 'executable',
    'process', 'message', 'proctitle'
]

# Create a copy of df with only those columns
df_new = df[fields_to_keep].copy()
df_new = df_new.where(pd.notna(df_new), None)


In [ ]:
df_new

In [ ]:
df_new.to_json('splunk-output.json', orient='records', lines=True)
